In [1]:
# Libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
import urllib
import urllib2
import re
import unicodedata
import datetime
import requests
import requests_cache
requests_cache.install_cache('demo_cache')
%matplotlib inline

In [47]:
results = pd.read_excel('data/nba_results.xlsx')

In [48]:
teams = []
for team in results['Team']:
    teams.append(str(''.join(x for x in team if x.isalpha() or x.isspace() or x.isnumeric())))

In [49]:
teams[:5]

['Boston Celtics',
 'Cleveland Cavaliers',
 'Toronto Raptors',
 'Washington Wizards',
 'Atlanta Hawks']

In [50]:
results['Team'] = teams

In [51]:
results.head()

,Team,W,L,WL_pct,PW,PL,PS/G,PA/G,yr,abbr,playoffs
0,Boston Celtics,53,29,0.646,48,34,108.0,105.4,2017,BOS,3
1,Cleveland Cavaliers,51,31,0.622,49,33,110.3,107.2,2017,CLE,4
2,Toronto Raptors,51,31,0.622,52,30,106.9,102.6,2017,TOR,2
3,Washington Wizards,49,33,0.598,46,36,109.2,107.4,2017,WAS,2
4,Atlanta Hawks,43,39,0.524,39,43,103.2,104.0,2017,ATL,1


In [53]:
results[results['Team'] == 'Brooklyn Nets']

,Team,W,L,WL_pct,PW,PL,PS/G,PA/G,yr,abbr,playoffs
14,Brooklyn Nets,20,62,0.244,24,58,105.8,112.5,2017,BKN,0
43,Brooklyn Nets,21,61,0.256,22,60,98.6,106.0,2016,BKN,0
62,Brooklyn Nets,38,44,0.463,33,49,98.0,100.9,2015,BKN,1
91,Brooklyn Nets,44,38,0.537,38,44,98.5,99.5,2014,BKN,2
121,Brooklyn Nets,49,33,0.598,46,36,96.9,95.1,2013,BKN,1


In [54]:
# Abbreviations for team names--gotta be a better way, but this works and is quick enough
def return_abbr(team):
    if team == 'San Antonio Spurs':
        return 'SAS'
    elif team == 'New York Knicks':
        return 'NYK'
    elif team == 'Brooklyn Nets':
        return 'BRK'
    elif team == 'New Jersey Nets':
        return 'NJN'
    elif team == 'Golden State Warriors':
        return 'GSW'
    elif team == 'Washington Bullets':
        return 'WSB'
    elif team == 'Los Angeles Lakers':
        return 'LAL'
    elif team == 'Los Angeles Clippers':
        return 'LAC'
    elif team == 'Oklahoma City Thunder':
        return 'OKC'
    elif team == 'New Orleans Pelicans':
        return 'NOP'
    elif team == 'New Orleans Hornets':
        return 'NOH'
    elif team == 'Charlotte Hornets':
        return 'CHH'
    else:
        return team[:3].upper()

In [55]:
abbr = []
for team in results['Team']:
    abbr.append(return_abbr(team))

In [56]:
abbr[:5]

['BOS', 'CLE', 'TOR', 'WAS', 'ATL']

In [57]:
results['abbr'] = abbr

In [59]:
results.to_excel('data/nba_results.xlsx')

In [60]:
pd.read_excel('data/nba_results.xlsx')

,Team,W,L,WL_pct,PW,PL,PS/G,PA/G,yr,abbr,playoffs
0,Boston Celtics,53,29,0.646,48,34,108.0,105.4,2017,BOS,3
1,Cleveland Cavaliers,51,31,0.622,49,33,110.3,107.2,2017,CLE,4
2,Toronto Raptors,51,31,0.622,52,30,106.9,102.6,2017,TOR,2
3,Washington Wizards,49,33,0.598,46,36,109.2,107.4,2017,WAS,2
4,Atlanta Hawks,43,39,0.524,39,43,103.2,104.0,2017,ATL,1
5,Milwaukee Bucks,42,40,0.512,40,42,103.6,103.8,2017,MIL,1
6,Indiana Pacers,42,40,0.512,40,42,105.1,105.3,2017,IND,1
7,Chicago Bulls,41,41,0.500,42,40,102.9,102.4,2017,CHI,1
8,Miami Heat,41,41,0.500,44,38,103.2,102.1,2017,MIA,0
9,Detroit Pistons,37,45,0.451,38,44,101.3,102.5,2017,DET,0
